In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
95,application_1638590815625_0096,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-95>

In [2]:
%%configure -f 
{
"conf":{
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    
        "spark.executor.heartbeatInterval":"10800s",
        "spark.network.timeout":"24h",
    
        "spark.driver.memory": "8G",
        "spark.executor.memory": "8G",
        "spark.executor.cores":"4",
    
        "livy.server.session.timeout" : "5h",

        "spark.app.name":"msds694"
      }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
96,application_1638590815625_0097,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
96,application_1638590815625_0097,pyspark,idle,Link,Link,✔


In [5]:
import time
start = time.time()
start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1638953765.3834593

marketplace

customer_id

review_id

productid

product_parent

product_title

product_category

star_rating

helpful_votes

total_votes

vine

verified_purchase

review_headline

review_body

review_date

In [3]:
from datetime import datetime
data_path = 's3://msds694-project-zhipeng/data'
rdd = sc.textFile(data_path, 8)
header=rdd.take(1)
rdd=rdd.filter(lambda x:x != header[0])
rdd=rdd.map(lambda x:x.split('\t'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
rdd.cache()
rdd.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['US', '33605939', 'RGYFDX8QXKEIR', 'B007KO2MLO', '328837464', 'Big Maria', 'Digital_Ebook_Purchase', '4', '0', '0', 'N', 'N', 'Quirky', 'Elmore Leonard meets the cast of Sierra Madre. Just a quirky read that will make you want to keep trying no matter what happens.', '2013-09-09']]

In [5]:
def time(x):
    x[-1]=datetime.strptime(x[-1][:7],'%Y-%m')
    return x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
rdd1 = rdd.map(lambda x: time(x))
count = rdd1.map(lambda x: ((x[6],x[-1]),1)).reduceByKey(lambda x,y: x+y)\
.map(lambda x: (x[0][0], x[0][1], x[1])).toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
count.createOrReplaceTempView('count_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%sql -o count_tab
select * from count_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%local
import plotly.graph_objects as go
import pandas as pd
count_tab = count_tab[count_tab['_2']>='2006-01']
count_tab = count_tab.sort_values('_2')
count_tab['_2'] = pd.to_datetime(count_tab['_2'])

fig = go.Figure()
colors = ['rgb(255,192,203)','rgb(49,130,189)', 'rgb(210,105,30)', 'rgb(46,139,87)',  'rgb(189,189,189)']
label = count_tab['_1'].unique()

for i in range(len(label)):
    color = colors[i]
    l = label[i]
    viz = count_tab[count_tab['_1']==l]
    fig.add_trace(go.Scatter(x= viz['_2'], y = viz['_3'], name = l, line = dict(color = color, width = 3), mode='lines'))

annotations = []

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
annotations = []

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Review Count for Each Category',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
# Legend
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=1.02,
    xanchor="right",
    x=1.3
))
fig.update_layout(annotations=annotations)

fig.show()

In [13]:
def mean(x):
    return sum(x)/len(x)
rdd2 = rdd.map(lambda x: time(x)).filter(lambda x: int(x[9])!=0)
helpvote= rdd2.map(lambda x: ((x[6],x[-1]), int(x[8])/int(x[9]) )).groupByKey().mapValues(lambda x: mean(x))\
                   .map(lambda x: (x[0][0], x[0][1], x[1])).toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%local
import plotly.graph_objects as go
import pandas as pd
def ts_cat_plot(data, title):
    fig = go.Figure()
    colors = ['rgb(255,192,203)','rgb(49,130,189)', 'rgb(210,105,30)', 'rgb(46,139,87)',  'rgb(189,189,189)']
    label = data['_1'].unique()

    for i in range(len(label)):
        color = colors[i]
        l = label[i]
        viz = data[data['_1']==l]
        fig.add_trace(go.Scatter(x= viz['_2'], y = viz['_3'], name = l, line = dict(color = color, width = 3), mode='lines'))

    annotations = []

    fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        autosize=False,
        margin=dict(
            autoexpand=True,
            l=100,
            r=20,
            t=110,
        ),
        showlegend=True,
        plot_bgcolor='white'
    )
    annotations = []

    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                  xanchor='left', yanchor='bottom',
                                  text=title,
                                  font=dict(family='Arial',
                                            size=25,
                                            color='rgb(37,37,37)'),
                                  showarrow=False))

    annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                                  xanchor='center', yanchor='top',
                                  text='Year',
                                  font=dict(family='Arial',
                                            size=12,
                                            color='rgb(150,150,150)'),
                                  showarrow=False))
    # Legend
    fig.update_layout(legend=dict(
        orientation="v",
        yanchor="top",
        y=1.02,
        xanchor="right",
        x=1.3
    ))
    fig.update_layout(annotations=annotations)

    fig.show()

In [15]:
helpvote.createOrReplaceTempView('helpvote_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%sql -o helpvote_tab
select * from helpvote_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [17]:
%%local 
helpvote_tab = helpvote_tab[helpvote_tab['_2']>='2006']
helpvote_tab = helpvote_tab.sort_values('_2')
helpvote_tab['_2'] = pd.to_datetime(helpvote_tab['_2'])
ts_cat_plot(helpvote_tab, 'Helpful Vote Ratio for Each Category')

In [18]:
rdd3 = rdd.map(lambda x: time(x))
def verify_purchase(x):
    if x == 'N': # 'N': Discount
        x = 1
    else: x =0 
    return x
review_cat= rdd3.map(lambda x: ((x[6],x[-1]), x[11])).mapValues(lambda x: verify_purchase(x))\
.groupByKey().mapValues(lambda x: mean(x)).map(lambda x: (x[0][0], x[0][1], x[1])).toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
review_cat.createOrReplaceTempView('verify_purchase_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
%%sql -o verify_purchase_tab
select * from verify_purchase_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [21]:
%%local 
verify_purchase_tab = verify_purchase_tab[verify_purchase_tab['_2']>='2006']
verify_purchase_tab = verify_purchase_tab.sort_values('_2')
verify_purchase_tab['_2'] = pd.to_datetime(verify_purchase_tab['_2'])
ts_cat_plot(verify_purchase_tab, 'Discount Purchase Ratio for Each Category')

In [22]:
rdd4 = rdd.map(lambda x: time(x))
def vote_click(x):
    if x != 0:
        x = 1
    return x

voteclick = rdd4.map(lambda x: ((x[6],x[-1]), int(x[9]))).mapValues(lambda x: vote_click(x))\
.groupByKey().mapValues(lambda x: mean(x)).map(lambda x: (x[0][0], x[0][1], x[1])).toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
voteclick.createOrReplaceTempView('voteclick_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
%%sql -o voteclick_tab
select * from voteclick_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [25]:
%%local 
voteclick_tab = voteclick_tab[voteclick_tab['_2']>='2006']
voteclick_tab = voteclick_tab.sort_values('_2')
voteclick_tab['_2'] = pd.to_datetime(voteclick_tab['_2'])
ts_cat_plot(voteclick_tab, 'Click Vote Ratio for Each Category')

In [26]:
rdd5 = rdd.map(lambda x: time(x))
total_count = rdd5.map(lambda x: (x[-1], 1)).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], x[1])).toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
total_count.createOrReplaceTempView('total_count_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
%%sql -o total_count_tab
select * from total_count_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%%local
total_count_tab = total_count_tab[total_count_tab['_1']>='2006']
total_count_tab = total_count_tab.sort_values('_1')
total_count_tab['_1'] = pd.to_datetime(total_count_tab['_1'])
fig = go.Figure()
fig.add_trace(go.Scatter(x = total_count_tab['_1'], y = total_count_tab['_2'], mode = 'lines', line = dict(width =3)))

annotations = []

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        type = 'log',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)
annotations = []

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Total Review Count',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
# Legend
fig.update_layout(annotations=annotations)

fig.show()

In [30]:
# execution time
import time
start - time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-209.88648867607117

In [31]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…